In [2]:
# ================================================================
# 1) INSTALL DEPENDENCIES
# ================================================================
!pip install -q transformers accelerate pillow torch torchvision sentencepiece scikit-image


In [3]:
# ================================================================
# 2) IMPORTS
# ================================================================
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from skimage import filters
import cv2
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import files


In [4]:
# ================================================================
# 3) LOAD BLIP (DESCRIPTION)
# ================================================================
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# ================================================================
# 4) LOAD GPT2 (HASHTAGS)
# ================================================================
tokenizer = AutoTokenizer.from_pretrained("gpt2")
hashtag_model = AutoModelForCausalLM.from_pretrained("gpt2")

# ================================================================
# 5) LOAD RESNET50 FOR QUALITY SCORE
# ================================================================
resnet = models.resnet50(pretrained=True)
resnet.fc = torch.nn.Identity()
resnet.eval()

res_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 140MB/s]


In [5]:
# ================================================================
# 6) SMART FEATURE EXTRACTION (NO DATASET NEEDED)
# ================================================================

def compute_colorfulness(img):
    (B, G, R) = cv2.split(img.astype("float"))
    rg = np.abs(R - G)
    yb = np.abs((R + G)/2 - B)
    std_root = np.sqrt(rg.var() + yb.var())
    mean_root = np.sqrt(rg.mean()**2 + yb.mean()**2)
    return std_root + 0.3 * mean_root

def compute_sharpness(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return filters.laplace(gray).var()

def compute_edge_density(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return edges.mean() / 255

def compute_brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    return hsv[:,:,2].mean()

def get_quality_embedding(img_pil):
    t = res_transform(img_pil).unsqueeze(0)
    with torch.no_grad():
        emb = resnet(t).squeeze().numpy()
    return float(np.linalg.norm(emb))

In [6]:
def detect_dominant_color(image_path):
    img = Image.open(image_path).convert("RGB")
    img_small = img.resize((50, 50))
    arr = np.array(img_small).reshape(-1, 3)

    # simple clustering-free dominant color
    avg = arr.mean(axis=0)
    r, g, b = avg

    if r > 150 and g < 100 and b < 100:
        return "red"
    if r > 180 and g > 150 and b < 100:
        return "orange"
    if r > 200 and g > 200 and b > 200:
        return "white"
    if r < 80 and g < 80 and b < 80:
        return "black"
    if r > 180 and g > 180 and b < 160:
        return "beige"
    if r > 120 and g < 100 and b < 80:
        return "brown"

    return "neutral"


In [31]:
def generate_product_description(category, material, color_desc, design_desc):
    material = material.capitalize()
    color_desc = color_desc.capitalize()
    design_desc = design_desc.lower()

    # Convert design text into natural language
    if "floral" in design_desc:
        design_phrase = "a beautifully crafted floral pattern"
    elif "pattern" in design_desc:
        design_phrase = "an intricately woven pattern"
    elif "stripe" in design_desc:
        design_phrase = "subtle stripe detailing"
    elif "texture" in design_desc:
        design_phrase = "a rich handcrafted textured surface"
    elif "embroidery" in design_desc:
        design_phrase = "detailed artisanal embroidery"
    elif "print" in design_desc:
        design_phrase = "a traditional hand-printed design"
    elif "motif" in design_desc:
        design_phrase = "cultural motif detailing"
    elif "carved" in design_desc:
        design_phrase = "hand-carved surface detailing"
    elif "plain" in design_desc:
        design_phrase = "a clean, plain minimalist finish"
    else:
        design_phrase = f"a {design_desc}"

    # ---------------------------------------------------------
    # POTTERY — LONG DESCRIPTION WITH DESIGN DETAILS
    # ---------------------------------------------------------
    if category == "pottery":
        return (
            f"This handcrafted {material} piece is shaped using traditional artisanal pottery techniques, ensuring authenticity and cultural value. "
            f"It features {design_phrase} enhanced by a warm {color_desc} finish that adds depth and natural character to the piece. "
            f"The artisan’s handwork is visible in every contour, allowing the design details to subtly complement the form and texture of the pottery. "
            f"Made from high-quality clay, the piece carries natural grain patterns and an organic feel that reflect slow, mindful craftsmanship. "
            f"The balanced structure, smooth finishing and design placement make it suitable for rustic, modern and earthy home interiors. "
            f"This décor piece serves beautifully as a centerpiece, shelf accent or gifting option for homes that value handcrafted beauty. "
            f"Perfect for décor enthusiasts looking for pieces that combine heritage craft, elegant design and long-lasting durability."
        )

    # ---------------------------------------------------------
    # FABRIC — LONG DESCRIPTION WITH DESIGN DETAILS
    # ---------------------------------------------------------
    if category == "clothing":
        return (
            f"This premium {material} fabric is handwoven on a traditional loom, where artisans meticulously integrate {design_phrase} into the textile structure. "
            f"The fabric carries a graceful {color_desc} shade that complements the detailed craftsmanship, giving it a refined and versatile appeal. "
            f"Its breathable weave, soft handfeel and natural drape make it ideal for tailoring sarees, stoles, kurtas, blouses and dressmaking projects. "
            f"The handcrafted design work enhances the textile’s premium quality, making it suitable for festive wear, ethnic fashion and contemporary styling. "
            f"The intricate weaving and design elements reflect artisanal skill rather than machine-produced uniformity—giving each piece a unique identity. "
            f"In home décor, this fabric works beautifully for curtains, cushion covers, table spreads and other creative projects. "
            f"An excellent choice for customers who appreciate sustainable fashion, detailed craftsmanship and culturally rooted handloom artistry."
        )

    # ---------------------------------------------------------
    # HANDBAGS — LONG DESCRIPTION WITH DESIGN DETAILS
    # ---------------------------------------------------------
    if category == "handbag":
        return (
            f"This handcrafted {material} handbag combines durability with artisan skill, featuring {design_phrase} that enhances its natural character. "
            f"The {color_desc} tone gives the bag a stylish and versatile appearance suitable for both ethnic and modern outfits. "
            f"Artisans carefully shape and finish the material, ensuring that the design elements integrate seamlessly with the bag’s structure. "
            f"The lightweight yet sturdy build ensures comfortable everyday use while preserving the aesthetic richness of handcrafted fashion. "
            f"With thoughtful detailing and spacious functionality, it is perfect for daily outings, casual styling or gifting. "
            f"An ideal accessory for conscious buyers who prefer eco-friendly materials, artisanal work and distinctive handcrafted design."
        )

    # ---------------------------------------------------------
    # WOODEN TABLE — LONG DESCRIPTION WITH DESIGN DETAILS
    # ---------------------------------------------------------
    if category == "table":
        return (
            f"This handcrafted {material} table is built by skilled carpenters who blend traditional woodworking techniques with modern design sensibilities. "
            f"It features {design_phrase} complemented by a warm {color_desc} finish that highlights the natural grains and patterns of the wood. "
            f"The precision craftsmanship ensures a sturdy structure, smooth edges and long-lasting durability. "
            f"The design detailing enhances its aesthetic value, making it suitable for living rooms, workspaces, studios and décor-focused spaces. "
            f"Whether used as a centerpiece, utility table or decorative furniture piece, it brings an elegant blend of nature and craftsmanship into your home. "
            f"Perfect for buyers who appreciate quality woodworking, artistic design and premium handmade furniture."
        )

    # ---------------------------------------------------------
    # PLATES — LONG DESCRIPTION WITH DESIGN DETAILS
    # ---------------------------------------------------------
    if category == "plate":
        return (
            f"This handcrafted {material} plate showcases {design_phrase} beautifully highlighted by a clean {color_desc} surface. "
            f"The artisan’s focus on detailing gives the plate a refined, premium appearance suitable for both everyday meals and special occasions. "
            f"Its smooth texture, balanced weight and heat-resistant build make it durable and functional for serving or dining. "
            f"The design placement enhances its aesthetic elegance, making it ideal for food styling, photography, gifting and home décor. "
            f"Perfect for customers who appreciate artisanal tableware that blends functionality with cultural craftsmanship."
        )

    # ---------------------------------------------------------
    # GENERIC — LONG DESCRIPTION WITH DESIGN DETAILS
    # ---------------------------------------------------------
    return (
        f"This handcrafted product is made from fine {material} and features {design_phrase} paired with a soothing {color_desc} aesthetic. "
        f"It carries a refined handmade character, showcasing artisan skill and natural material beauty. "
        f"The detailed design work adds personality and artistic value, making it suitable for décor, styling or everyday use. "
        f"An excellent choice for buyers who appreciate craftsmanship, thoughtful design and premium handcrafted quality."
    )


In [32]:
def generate_product_details(material, color_desc, design_desc, category):
    # ----------------------------
    # CLOTHING / FABRIC
    # ----------------------------
    if category == "clothing":
        highlighted_points = [
            f"• Material: {material}",
            f"• Colour: {color_desc}",
            f"• Design: {design_desc}",
            "• Soft, breathable & skin-friendly",
            "• Woven by skilled artisan weavers"
        ]

    # ----------------------------
    # POTTERY / TERRACOTTA
    # ----------------------------
    elif category == "pottery":
        highlighted_points = [
            f"• Material: {material}",
            f"• Colour: {color_desc}",
            f"• Finish: {design_desc}",
            "• Handcrafted using traditional pottery techniques",
            "• Durable, eco-friendly & artisan-made"
        ]

    # ----------------------------
    # HANDBAGS (jute, leather, fabric, cane, etc.)
    # ----------------------------
    elif category == "handbag":
        highlighted_points = [
            f"• Material: {material}",
            f"• Colour: {color_desc}",
            f"• Design: {design_desc}",
            "• Lightweight, durable & comfortable to carry",
            "• Perfect for daily use, outings & ethnic styling"
        ]

    # ----------------------------
    # WOODEN TABLES / FURNITURE
    # ----------------------------
    elif category == "table":
        highlighted_points = [
            f"• Material: {material}",
            f"• Colour: {color_desc}",
            f"• Finish: {design_desc}",
            "• Strong, sturdy & long-lasting build",
            "• Handcrafted by skilled carpenters"
        ]

    # ----------------------------
    # PLATES (ceramic / clay / metal)
    # ----------------------------
    elif category == "plate":
        highlighted_points = [
            f"• Material: {material}",
            f"• Colour: {color_desc}",
            f"• Surface Finish: {design_desc}",
            "• Food-safe, heat-resistant & durable",
            "• Perfect for dining, serving & gifting"
        ]

    # ----------------------------
    # DEFAULT FALLBACK
    # ----------------------------
    else:
        highlighted_points = [
            f"• Material: {material}",
            f"• Colour: {color_desc}",
            f"• Finish: {design_desc}",
            "• Artisan-crafted & eco-friendly",
            "• Suitable for décor, gifting & everyday use"
        ]

    return highlighted_points


In [33]:
def generate_hashtags(category, material, color_desc, design_desc):
    hashtags = []

    # Normalize
    mat = material.lower()
    col = color_desc.lower()
    des = design_desc.lower()
    cat = category.lower()

    # -----------------------------
    # CATEGORY-BASED TAGS
    # -----------------------------
    if cat == "pottery":
        hashtags += [
            "#pottery", "#clayart", "#terracotta", "#earthenware",
            "#handcraftedpottery", "#artisanpottery"
        ]

    elif cat == "clothing":
        hashtags += [
            "#handloom", "#wovenfabric", "#textileart", "#artisanfabric",
            "#sustainablefashion", "#ethicalfashion"
        ]

    elif cat == "handbag":
        hashtags += [
            "#handmadebag", "#artisanbag", "#ecofriendlybag",
            "#craftbag", "#sustainableaccessories"
        ]

    elif cat == "table":
        hashtags += [
            "#woodcraft", "#handmadetable", "#carpentryart",
            "#woodenfurniture", "#homedecorfurniture"
        ]

    elif cat == "plate":
        hashtags += [
            "#handcraftedplates", "#ceramicart", "#tableware",
            "#kitchendecor", "#diningessentials"
        ]

    else:
        hashtags += ["#handcrafted", "#artisanmade"]

    # -----------------------------
    # MATERIAL TAGS
    # -----------------------------
    material_map = {
        "cotton": "#cottonfabric",
        "linen": "#linenfabric",
        "silk": "#silkweaves",
        "wool": "#woolfabric",
        "terracotta": "#terracottacraft",
        "clay": "#claycraft",
        "ceramic": "#ceramicware",
        "wood": "#woodcraft"
    }

    for key, tag in material_map.items():
        if key in mat:
            hashtags.append(tag)

    # -----------------------------
    # COLOR TAGS
    # -----------------------------
    color_map = {
        "brown": "#browndecor",
        "beige": "#beigenatural",
        "red": "#redtextile",
        "orange": "#orangecraft",
        "white": "#whitedecor",
        "black": "#blackaesthetic",
        "neutral": "#naturaltone"
    }

    for key, tag in color_map.items():
        if key in col:
            hashtags.append(tag)

    # -----------------------------
    # DESIGN TAGS
    # -----------------------------
    design_map = {
        "plain": "#minimaldesign",
        "pattern": "#patternedcraft",
        "texture": "#texturedfabric",
        "smooth": "#smoothfinish",
        "stripe": "#stripeddesign"
    }

    for key, tag in design_map.items():
        if key in des:
            hashtags.append(tag)

    # -----------------------------
    # UNIVERSAL ARTISAN TAGS
    # -----------------------------
    hashtags += [
        "#handmade", "#indiancrafts", "#vocalforlocal",
        "#madeinindia", "#traditionalcraft", "#supportlocalartisans",
        "#homedecor", "#artisanal", "#craftcommunity"
    ]

    # Remove duplicates
    hashtags = list(dict.fromkeys(hashtags))

    # Limit to 10 clean hashtags
    return hashtags[:10]


In [34]:
# ================================================================
# 9) INTELLIGENT PRICE SCORING MODEL (NO DATASET)
# ================================================================
def predict_price(path):
    pil = Image.open(path).convert("RGB")
    img = np.array(pil)

    # Extract features
    color = compute_colorfulness(img)
    sharp = compute_sharpness(img)
    edges = compute_edge_density(img)
    bright = compute_brightness(img)
    quality = get_quality_embedding(pil)

    # Normalize features
    color_s = color / 20
    sharp_s = sharp / 200
    edges_s = edges * 5
    bright_s = bright / 255
    quality_s = quality / 2000

    # Weighted scoring system
    score = (
        0.28 * color_s +
        0.22 * sharp_s +
        0.20 * edges_s +
        0.15 * bright_s +
        0.35 * quality_s
    )

    # Convert score → price
    base = 300  # base for handmade Indian products
    multiplier = (score * 1800)

    price = base + multiplier

    # Keep it clean
    price = max(120, price)
    price = int(round(price / 10) * 10)

    return price


In [35]:
def detect_material(caption):
    caption = caption.lower()

    # DEFAULT
    material = "handcrafted material"
    mat_type = "generic"

    # ------------------------------
    # POTTERY / CLAY MATERIALS
    # ------------------------------
    pottery_map = {
        "terracotta": ("Terracotta Clay", "clay"),
        "clay": ("Natural Clay", "clay"),
        "earthenware": ("Earthenware Clay", "clay"),
        "ceramic": ("Ceramic Material", "ceramic"),
        "porcelain": ("Porcelain Ceramic", "ceramic"),
        "stoneware": ("Stoneware Clay", "clay")
    }

    for key, (mat, mtype) in pottery_map.items():
        if key in caption:
            return mat, mtype

    # ------------------------------
    # FABRIC / TEXTILE MATERIALS
    # ------------------------------
    fabric_map = {
        "cotton": ("Pure Cotton Fabric", "textile"),
        "linen": ("Premium Linen Fabric", "textile"),
        "silk": ("Soft Silk Fabric", "textile"),
        "wool": ("Fine Woolen Fabric", "textile"),
        "khadi": ("Handspun Khadi Cotton", "textile"),
        "polyester": ("Polyester Fabric", "synthetic textile"),
        "synthetic": ("Synthetic Fabric", "synthetic textile"),
        "fabric": ("Woven Handloom Fabric", "textile"),
        "textile": ("Handloom Textile", "textile")
    }

    for key, (mat, mtype) in fabric_map.items():
        if key in caption:
            return mat, mtype

    # ------------------------------
    # NATURAL FIBER MATERIALS (bags, baskets, decor)
    # ------------------------------
    natural_fiber_map = {
        "jute": ("Jute Fiber", "natural fiber"),
        "cane": ("Cane Weave", "natural fiber"),
        "bamboo": ("Bamboo Weave", "natural fiber"),
        "wicker": ("Wicker Weave", "natural fiber"),
        "straw": ("Straw Fiber", "natural fiber"),
        "rattan": ("Rattan Fiber", "natural fiber")
    }

    for key, (mat, mtype) in natural_fiber_map.items():
        if key in caption:
            return mat, mtype

    # ------------------------------
    # WOOD MATERIALS (tables, decor)
    # ------------------------------
    wood_map = {
        "wood": ("Natural Wood", "hardwood"),
        "wooden": ("Handcrafted Wood", "hardwood"),
        "teak": ("Teak Wood", "hardwood"),
        "sheesham": ("Sheesham Wood", "hardwood"),
        "mango wood": ("Mango Wood", "hardwood")
    }

    for key, (mat, mtype) in wood_map.items():
        if key in caption:
            return mat, mtype

    # ------------------------------
    # METAL MATERIALS (plates, décor, accessories)
    # ------------------------------
    metal_map = {
        "brass": ("Brass Metal", "metal"),
        "copper": ("Copper Metal", "metal"),
        "iron": ("Iron Metal", "metal"),
        "steel": ("Stainless Steel", "metal"),
        "aluminium": ("Aluminium Metal", "metal"),
        "bronze": ("Bronze Metal", "metal"),
        "metal": ("Metal Alloy", "metal")
    }

    for key, (mat, mtype) in metal_map.items():
        if key in caption:
            return mat, mtype

    # ------------------------------
    # LEATHER MATERIALS (bags)
    # ------------------------------
    leather_map = {
        "leather": ("Genuine Leather", "leather"),
        "vegan leather": ("Vegan Leather", "leather"),
        "faux leather": ("Faux Leather", "leather")
    }

    for key, (mat, mtype) in leather_map.items():
        if key in caption:
            return mat, mtype

    # ------------------------------
    # CERAMIC / PLATE-SPECIFIC MATERIALS
    # ------------------------------
    if "plate" in caption or "dish" in caption:
        if "ceramic" in caption:
            return "Ceramic Plate", "ceramic"
        if "clay" in caption or "terracotta" in caption:
            return "Terracotta Clay Plate", "clay"
        if "steel" in caption:
            return "Stainless Steel Plate", "metal"
        if "brass" in caption:
            return "Brass Plate", "metal"

    # ------------------------------
    # FALLBACK
    # ------------------------------
    return material, mat_type


In [61]:
def detect_category(caption):
    caption = caption.lower().strip()

    # ----------------------------------------
    # 1) POTTERY / VASE / TERRACOTTA
    # ----------------------------------------
    pottery_keywords = [
        "pot", "vase", "jar", "pitcher", "jug",
        "pottery", "terracotta", "clay", "earthen", "earthenware",
        "ceramic", "mud pot", "handmade pot", "clay pot"
    ]
    if any(k in caption for k in pottery_keywords):
        return "pottery"

    # ----------------------------------------
    # 2) SUPER-STRONG CLOTHING DETECTION
    # ----------------------------------------

    # direct clothing/fabric words
    fabric_keywords = [
        "fabric", "cloth", "textile", "material", "woven",
        "handloom", "fiber", "cotton", "linen", "silk", "wool",
        "khadi", "garment", "scarf", "shawl", "dupatta",
        "stole", "saree", "sari", "kurta", "dress",
    ]
    if any(k in caption for k in fabric_keywords):
        return "clothing"

    # saree-specific indicators
    saree_keywords = [
        "pallu", "border", "drape", "pallu design", "zari", "weave pattern"
    ]
    if any(k in caption for k in saree_keywords):
        return "clothing"

    # BLIP generic phrases that ALWAYS indicate fabric
    strong_fabric_phrases = [
        "folded piece", "folded", "piece of cloth", "soft texture",
        "textured surface", "thin material", "on a surface",
        "lying cloth", "plain cloth", "sheet of fabric"
    ]
    if any(p in caption for p in strong_fabric_phrases):
        return "clothing"

    # sometimes BLIP says “a textured object” or “smooth soft object”
    texture_indicators = [
        "soft", "smooth", "textured", "wrinkled", "creased", "woven texture"
    ]
    if any(t in caption for t in texture_indicators):
        return "clothing"

    # if BLIP says "brown cloth", "white cloth"
    if "cloth" in caption:
        return "clothing"

    # if caption mentions color + surface -> usually fabric
    if any(color in caption for color in ["beige", "cream", "white", "off white", "brown", "tan"]):
        if "surface" in caption or "piece" in caption or "lying" in caption:
            return "clothing"

    # ----------------------------------------
    # 3) HANDBAGS / PURSES / TOTE BAGS
    # ----------------------------------------
    handbag_keywords = [
        "bag", "handbag", "purse", "tote", "jute bag",
        "shopping bag", "shoulder bag", "sling bag"
    ]
    if any(k in caption for k in handbag_keywords):
        return "handbag"

    # ----------------------------------------
    # 4) TABLE / WOODEN FURNITURE
    # ----------------------------------------
    table_keywords = [
        "table", "desk", "furniture", "coffee table", "dining table"
    ]
    if any(k in caption for k in table_keywords):
        return "table"

    # ----------------------------------------
    # 5) PLATES / DISHES / BOWLS
    # ----------------------------------------
    plate_keywords = [
        "plate", "dish", "bowl", "dinner plate", "ceramic plate",
        "clay plate", "serving plate", "tableware"
    ]
    if any(k in caption for k in plate_keywords):
        return "plate"

    # ----------------------------------------
    # 6) SMART FALLBACK LOGIC
    # ----------------------------------------

    # Soft/flat/woven appearance → fabric
    if any(w in caption for w in ["soft", "flat", "wrinkled", "woven", "sheet"]):
        return "clothing"

    # Thin, long, folded → fabric
    if "long" in caption or "thin" in caption:
        if "object" in caption or "piece" in caption:
            return "clothing"

    # If NOTHING else matches but caption sounds like material → fabric
    if "material" in caption:
        return "clothing"

    # ----------------------------------------
    # DEFAULT
    # ----------------------------------------
    return "Handmade product "


In [54]:
def generate_product_title(category):
    return f"**{category.capitalize()}**"


In [67]:
# ================================================================
# 10) UPLOAD IMAGE + RUN FULL PIPELINE (FINAL VERSION)
# ================================================================
print("Upload any product image (pottery, fabric, handbag, table, plate)…")
uploaded = files.upload()
path = list(uploaded.keys())[0]

# 1) Detect caption, category, material, color, design
img = Image.open(path).convert("RGB")
inputs = processor(img, return_tensors="pt")
out = blip.generate(**inputs, max_length=70)
caption = processor.decode(out[0], skip_special_tokens=True).lower()

category = detect_category(caption)
material, mat_type = detect_material(caption)
color = detect_dominant_color(path)

color_words = {
    "brown": "Earthy Brown",
    "beige": "Soft Beige",
    "red": "Deep Red",
    "orange": "Warm Orange",
    "white": "Clean White",
    "black": "Deep Black",
    "neutral": "Natural Toned"
}
color_desc = color_words.get(color, "Natural Toned")

# DESIGN DETECTION
if "plain" in caption:
    design_desc = "Plain Minimalist Finish"
elif "pattern" in caption:
    design_desc = "Subtle Woven Pattern"
elif "texture" in caption:
    design_desc = "Textured Handcrafted Surface"
elif "striped" in caption:
    design_desc = "Stripe Detailing"
else:
    design_desc = "Smooth Clean Handcrafted Finish"

# 2) Generate DESCRIPTION (new style)
description = generate_product_description(category, material, color_desc, design_desc)

# 3) Generate HIGHLIGHTED POINTS
highlighted_points = generate_product_details(material, color_desc, design_desc, category)

# 4) Generate HASHTAGS
hashtags = generate_hashtags(category, material, color_desc, design_desc)

# 5) Predict PRICE
price = predict_price(path)

# ============================
#          OUTPUT
# ============================
print("\n=======================")
title = generate_product_title(category)
print("TITLE:\n", title)

print("DESCRIPTION:\n")
print(description)

print("\nHIGHLIGHTED POINTS:\n")
for pt in highlighted_points:
    print(pt)

print("\nHASHTAGS:\n")
print(hashtags)

print("\nPREDICTED PRICE: ₹", price)
print("=======================\n")


Upload any product image (pottery, fabric, handbag, table, plate)…


Saving pot1.png to pot1 (1).png

TITLE:
 **Pottery**
DESCRIPTION:

This handcrafted Handcrafted material piece is shaped using traditional artisanal pottery techniques, ensuring authenticity and cultural value. It features a smooth clean handcrafted finish enhanced by a warm Natural toned finish that adds depth and natural character to the piece. The artisan’s handwork is visible in every contour, allowing the design details to subtly complement the form and texture of the pottery. Made from high-quality clay, the piece carries natural grain patterns and an organic feel that reflect slow, mindful craftsmanship. The balanced structure, smooth finishing and design placement make it suitable for rustic, modern and earthy home interiors. This décor piece serves beautifully as a centerpiece, shelf accent or gifting option for homes that value handcrafted beauty. Perfect for décor enthusiasts looking for pieces that combine heritage craft, elegant design and long-lasting durability.

HIGHLIG